

# Pipeline Objective
___
    * Capture images from file or video
    * Find Lane Markings in that Image
    * Annotate region of image containing lane markings with raw/hough lane markings
    * Extrapolate Line/Curve from the lane marking
    * Annotate image with extrapolated curve and return 

![alt text]( ./test_images/solidWhiteRight_annotated.jpg "Example Annotation")
___
# Reflection
Please find below the Various guiding principles and techniques used for the Pipeline to find lane lines on road surface.


## Guiding Principles
* Lane Markings in an image should be discriminable from the background based on color information. Since HSV space is a better representation of color space than RGB, Captured image should be converted to HSV space.
* Lane Markings in an image should be present in a defined region defined as trapezoid wide at the bottom of the image and narrowing as we go towards the top
* Edge Extraction and hough transform will provide us with a good set of lines that will either have positive or negative slope. Negative slope will represent lane markings to the left of the point of view while positive slope will represent right side lanes
* A first order polynomial fit on the line segments of each slope type should give us good extrapolated lane Markings


## 1. Pipeline Design

    1. Input Image -> hsv conversion -> white and yellow color segmentation -> color lane pixel candidates
    2. Input Image -> region discrimination -> gray scale conversion -> Edge Extraction -> Edge Pixels candidates
    3. Combining Color Candidates and Edge Candidates -> convert to grayscale as valid input for hough transform
    4. Combined Candidate -> Apply Hough Transform -> Lines
    5. Separate lines into positive and negative slopes and fit lines for each to get continuous lane representation
    6. Annotate original image with continuous lines
___

<img src="files/PipelineDescription.png">,

___
## 2. Learnings and Findings

* Color discrimination
1. HSV space color based segmentation was evaluated and subjectively found to give much better results in the presence of varying illumination as opposed to RGB image segmentation.
* Edge Extraction
2. Canny Edge Extraction parameters: reducing the thresholds caused a lot of spurious edges being detected and a noisy edge image is detected which is not useful. Increasing the thresholds beyond a certain point cause lane markings to be missed entirely over long term
3. Modifications to the draw_lines function: The list of lines returned by opencvs hough lines function was segregated by slope as positive vs negative slopes.. a small dead zone of slope was chosen (-0.2 > slope > 0.2) to eliminate positive sloped lines entering the negative cluster and vice versa. Then a single line was fit to all the x and y co-ordinates of each line type and then plotted.  

## 3. Shortcomings


1. I'm relying on slopes to segregate lines. What would happen When we're crossing the lane and there is only one lane in the field of view
2. Linear fitting to the lines yield poor results on images in challenge.mp4. This is possibly due to lane curvature as well as illumination artifacts


## 4. Improvements Planned
1. Try to fit and draw a quadratic polynomial to lane markers to account for curved lanes
2. Implement preprocessing steps to mitigate the effect of sunlight, shadow, reflections and other lighting artifacts.
3. Implement a pre-detection Inverse perspective projection step to extract lanes. I feel that inverse perspective mapping the lane regions to a top view will improve the lane detection accuracy both in color as well as edge space
4. Track lane markings using a bayesian filter like Kalman to mitigate effect of noise, missing lane markings et al. Between any two frames the deviation in the lane markers with respect to the point of view is very small. A KF/EKF should be filter should be able to track the lane markers well so as to make the pipeline more robust to failures/ inconsistencies in the detection step.
